In [4]:
!pip install regex


  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 16.9 MB/s eta 0:00:0000:01


In [65]:
import pandas as pd
import regex as re

movies = pd.read_csv("movies.csv")

In [66]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [67]:
def clean_title(title):
    title = re.sub(r"[^\w\s]", "", title)
    return title


In [68]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [69]:
movies.head()

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf(titles):
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    tfidf_matrix = vectorizer.fit_transform(titles)
    return tfidf_matrix

tfidf = compute_tfidf(movies["clean_title"])


In [71]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search_movie_by_title(title):
    # Limpa o título
    title_cleaned = clean_title(title)
    
    # Transforma o título limpo em um vetor TF-IDF
    query_vec = vectorizer.transform([title_cleaned])
    
    # Calcula a similaridade de cosseno entre o título da consulta e todos os outros títulos
    similarity_scores = cosine_similarity(query_vec, tfidf).flatten()
    
    # Pega os índices dos 5 filmes mais similares
    top_indices = np.argsort(similarity_scores)[-5:][::-1]
    
    # Retorna os resultados com base nos índices
    top_movies = movies.iloc[top_indices]
    
    return top_movies


In [72]:
import ipywidgets as widgets
from IPython.display import display

def interactive_movie_search():
    # Widget de entrada de texto
    movie_input = widgets.Text(
        value='Toy Story',
        description='Movie Title:',
        disabled=False
    )
    
    # Widget de saída para exibir os resultados
    movie_list = widgets.Output()

    # Função chamada sempre que o valor do widget de entrada muda
    def on_type(data):
        with movie_list:
            movie_list.clear_output()
            title = data["new"]
            # Pesquise filmes apenas se o título tiver mais de 5 caracteres
            if len(title) > 5:
                display(search_movie_by_title(title))

    # Observe mudanças no valor do widget de entrada
    movie_input.observe(on_type, names='value')

    # Exibe a interface interativa
    display(movie_input, movie_list)

# Chamada da função para exibir a interface
interactive_movie_search()


Text(value='Toy Story', description='Movie Title:')

Output()

In [73]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [74]:
ratings = pd.read_csv("ratings.csv")

In [75]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

## Função: `get_movie_recommendations`

A função `get_movie_recommendations` foi projetada para fornecer recomendações de filmes com base em dois critérios principais:

### 1. Usuários Semelhantes

- Inicialmente, a função identifica usuários que deram uma avaliação alta (maior que 4) para um filme específico (identificado pelo `movieId`).
- Esses usuários são considerados "semelhantes" por terem apreciado o mesmo filme que está sendo analisado.

### 2. Recomendações de Usuários Semelhantes vs. Todos os Usuários

- Para cada usuário semelhante, a função coleta os filmes que eles avaliaram positivamente.
- A proporção de usuários semelhantes que recomendaram cada filme é calculada.
- Filmes que são recomendados por uma proporção significativa de usuários semelhantes (mais de 10% neste caso) são filtrados.
- A função, então, avalia a popularidade desses filmes entre todos os usuários, fornecendo uma comparação entre as recomendações dos usuários semelhantes e do público geral.

### Resultado:

- A saída é um DataFrame que mostra as porcentagens de recomendações para filmes com base em usuários semelhantes (coluna `similar`) e todos os usuários (coluna `all`).
- Isso oferece uma visão sobre quais filmes são populares entre os usuários que gostaram do filme original e quais são populares entre todos os usuários.


In [76]:
def get_movie_recommendations(movie_id, ratings):
    # Encontrando usuários similares que deram alta avaliação para o filme especificado
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    
    # Recomendações de usuários similares
    similar_user_recs = ratings[ratings["userId"].isin(similar_users) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    
    # Filtrando recomendações com base em um limite
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    # Recomendações de todos os usuários para filmes recomendados por usuários similares
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    # Concatenando as recomendações de usuários similares e todos os usuários
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1, sort=False)
    rec_percentages.columns = ["similar", "all"]
    
    return rec_percentages

# Testando a função com o movieId 296
recommendations_final_test = get_movie_recommendations(296, ratings)
recommendations_final_test


,similar,all
movieId,,
296,1.000000,0.290095
318,0.492348,0.349394
593,0.396251,0.230073
2959,0.394234,0.218027
50,0.382489,0.205500
...,...,...
2502,0.101673,0.060401
904,0.101198,0.065907
1080,0.101198,0.056443


In [77]:
def get_top_scored_movies(rec_percentages, movies):
    # Calculando a coluna "score"
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    
    # Ordenando o DataFrame com base no "score" em ordem decrescente
    rec_percentages_sorted = rec_percentages.sort_values("score", ascending=False)
    
    # Unindo os 10 principais filmes com o DataFrame 'movies'
    top_movies = rec_percentages_sorted.head(10).merge(movies, left_index=True, right_on="movieId")
    
    return top_movies

In [78]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

## Interface de Recomendação de Filmes

O código apresentado cria uma interface interativa para recomendar filmes com base no título de um filme inserido pelo usuário. Aqui está uma descrição detalhada:

### Componentes:

#### 1. `movie_name_input`:
Um widget de entrada de texto onde o usuário pode digitar o nome de um filme.

#### 2. `recommendation_list`:
Um widget de saída que exibirá a lista de filmes recomendados.

### Funcionalidade:

- Quando um usuário começa a digitar o nome de um filme no widget `movie_name_input`, a função `on_type` é acionada.

- Dentro da função `on_type`:
  1. A lista de recomendações é limpa a cada novo caractere digitado.
  2. Se o comprimento do título inserido for maior que 5 caracteres, a função busca pelo título do filme usando `search_movie_by_title`.
  3. O ID do primeiro filme retornado pela busca é usado para encontrar filmes semelhantes com a função `find_similar_movies`.
  4. Os filmes recomendados são exibidos na área de saída `recommendation_list`.

### Uso:

1. Insira o título de um filme no widget `movie_name_input`.
2. À medida que você digita, uma lista de filmes recomendados será exibida em tempo real abaixo da caixa de entrada, com base nos filmes semelhantes ao título inserido.

---

Este código fornece uma maneira intuitiva e em tempo real para os usuários obterem recomendações de filmes simplesmente digitando o nome de um filme de interesse. O sistema retorna filmes semelhantes com base no conjunto de dados subjacente.


In [83]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search_movie_by_title(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='', description='Movie Title:')

Output()